In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

from data_generator import DataGenerator

import model as model_util


import matplotlib.pyplot as plt

from numpy import array
import numpy as np


import datetime
from datetime import datetime

from tensorflow.keras.layers import GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences


from tensorflow.keras import datasets, layers, models

import tensorflow as tf

#%load_ext tensorboard
print(tf.__version__)

2.0.0-rc0


In [2]:
data_gen = DataGenerator(random=False, first_index=10)
data_gen.rewind()
data_gen.steps

1008094

In [3]:
from tqdm import tqdm

data_gen.rewind()

path = "./"
import data_util

trainX, trainY, valX, valY = data_util.get_sets(data_gen, data_gen.steps - 2000, val_percentage = 0.003, path = path)



  0%|          | 72/1003075 [00:02<14:01:30, 19.87it/s]

2019-08-11 12:32:05 :  11350.85  ====  11351.985085  =====  2019-08-11 12:34:05  =====  11358.29


  0%|          | 433/1003075 [00:17<34:14:34,  8.13it/s]WARNING: Logging before flag parsing goes to stderr.
E0902 14:47:23.889365 140593921140544 ultratb.py:155] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/samir/anaconda3/envs/tf2/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-f7357013da87>", line 8, in <module>
    trainX, trainY, valX, valY = data_util.get_sets(data_gen, data_gen.steps - 2000, val_percentage = 0.003, path = path)
  File "/home/samir/Desktop/YaStockRnn/data_util.py", line 33, in get_sets
    trainX, trainY = get_set("train", int(data_count*(1-val_percentage)), data_gen,  path)
  File "/home/samir/Desktop/YaStockRnn/data_util.py", line 22, in get_set
    state = stateUtil.get_state(state, data_gen)
  File "/home/samir/Desktop/YaStockRnn/state_util.py", line 86, in get_state
    furure_state = self.get_future_state(data_gen)
  File "/home/samir/Desktop/YaStockRnn/state_util.py", line 70, in get_future_state
    state = data_gen.get_json_from_timestamp(timestamp_limit + index)
  File "/home/samir/Desktop/YaSt

KeyboardInterrupt: 

In [ ]:
# define generator
n_input = 100
def get_gen(set_x, set_y, shuffle=True):
    return TimeseriesGenerator(set_x, set_y, length=n_input, batch_size=64, shuffle=shuffle)

train_generator = get_gen(trainX, trainY)

val_generator = get_gen(valX, valY, shuffle=False)

# number of samples
print('Samples: %d' % len(train_generator))

x, y = train_generator[0]

features = x.shape[2]

print(features)

In [ ]:

model = model_util.get_model(n_input, features)

filepath = "stock.h5"


model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001, epsilon=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
tf.keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

In [ ]:


checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             load_weights_on_restart=True)

callbacks_list = [checkpoint]

history = model.fit_generator(train_generator, 
                              validation_data=val_generator, 
                              validation_steps=len(val_generator)-1,
                              steps_per_epoch=len(train_generator)-1, 
                              epochs=10, verbose=1, 
                              callbacks=callbacks_list)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
import datetime
print (datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))

In [ ]:
model_loaded = model_util.load_model(filepath)

In [ ]:

x, y = val_generator[8]
x, y = train_generator[8]

trainX = []
trainY = []

for i in range(len(val_generator)):
  x, y = train_generator[i] 
  loss, acc = model_loaded.evaluate(x, y)
  if (acc < 0.50):
    trainX.append(x)
    trainY.append(y)
    print("Accuracy: {:5.2f}%".format(100*acc))

In [ ]:
np.array(trainX).shape

In [1]:
for i in range(len(trainX)):
  xx = trainX[i]
  yy = trainY[i]
  model_loaded.fit(xx, yy, epochs=5, verbose=0)

NameError: name 'trainX' is not defined

In [3]:
from bitstamp import BitStamp

In [5]:
bt = BitStamp()

In [6]:
bt.start

<bound method BitStamp.start of <bitstamp.BitStamp object at 0x7f01889bbd30>>